In [1]:
!nvidia-smi

Thu Oct  3 10:46:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
%pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.8 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660544 sha256=1d890b4d7d7c4802ed9f198d1315ebe47cdfadb83113878ed42fa81b8219cce0
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [15]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import time

# CUDA-код для умножения матриц
matrix_mul_kernel = """
__global__ void matrixMulKernel(float* a, float* b, float* c, int rows_a, int cols_a, int cols_b) {
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Индекс строки в результирующей матрице
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Индекс столбца в результирующей матрице

    if (row < rows_a && col < cols_b) {
        float value = 0.0f;

        // Вычисление элемента результирующей матрицы
        for (int k = 0; k < cols_a; k++) {
            value += a[row * cols_a + k] * b[k * cols_b + col];
        }
        c[row * cols_b + col] = value; // Запись результата в итоговую матрицу
    }
}
"""

def matrix_multiply_cpu(A, B):
    '''
    Функция для выполнения умножения матриц на CPU
    :param: A - первая матрица
    :param: B - вторая матрица
    :return: [
        answer - результат умножения,
        время выполнения операции
    ]
    '''
    if A.shape[1] != B.shape[0]:
        raise ValueError("Неправильные размеры матриц для перемножения")

    answer = np.zeros((A.shape[0], B.shape[1]))

    start_time = time.time()
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                answer[i, j] += A[i, k] * B[k, j]
    end_time = time.time()

    return answer, end_time - start_time

def matrix_multiply_gpu(A, B):
    '''
    Функция для выполнения умножения матриц на GPU
    :param: A - первая матрица
    :param: B - вторая матрица
    :return: [
        C - результат умножения,
        время выполнения операции
    ]
    '''
    if A.shape[1] != B.shape[0]:
        raise ValueError("Неправильные размеры матриц для перемножения")

    A_height, A_width = A.shape
    B_height, B_width = B.shape
    C = np.zeros((A_height, B_width), dtype=np.float32)

    start_time = time.time()

    # Выделение памяти на GPU
    A_gpu = cuda.mem_alloc(A.nbytes)
    B_gpu = cuda.mem_alloc(B.nbytes)
    C_gpu = cuda.mem_alloc(C.nbytes)

    # Копирование данных на GPU
    cuda.memcpy_htod(A_gpu, A)
    cuda.memcpy_htod(B_gpu, B)

    # Компиляция и загрузка CUDA-кода
    mod = SourceModule(matrix_mul_kernel)
    matrix_mul = mod.get_function("matrixMulKernel")

    # Определяем размеры блока и сетки для распараллеливания
    block_size = (16, 16, 1)
    grid_size = ((B_width + block_size[0] - 1) // block_size[0],
                  (A_height + block_size[1] - 1) // block_size[1])

    # Запуск ядра на GPU
    matrix_mul(A_gpu, B_gpu, C_gpu, np.int32(A_height), np.int32(A_width), np.int32(B_width), block=block_size, grid=grid_size)

    cuda.Context.synchronize()

    # Копирование результата с GPU на CPU
    cuda.memcpy_dtoh(C, C_gpu)

    end_time = time.time()

    return C, end_time - start_time

if __name__ == "__main__":
    size = (2000, 2000)
    low = 1
    high = 10
    matrix_a = np.random.randint(low, high, size).astype(np.float32)
    matrix_b = np.random.randint(low, high, size).astype(np.float32)

    # Сравнение с результатом последовательного умножения
    answer_function = np.dot(matrix_a, matrix_b)

    # Умножение на GPU
    answer_gpu, time_gpu = matrix_multiply_gpu(matrix_a, matrix_b)
    print(f"Результат на GPU:\n{answer_gpu}")
    print(f"Соответствие результата с np.dot(): {np.allclose(answer_gpu, answer_function)}")
    print(f"Время на GPU: {time_gpu} секунд")

    # Умножение на CPU
    answer_cpu, time_cpu = matrix_multiply_cpu(matrix_a, matrix_b)
    print(f"Результат на CPU:\n{answer_cpu}")
    print(f"Соответствие результата с np.dot(): {np.allclose(answer_cpu, answer_function)}")
    print(f"Время на CPU: {time_cpu} секунд")


Результат на GPU:
[[49437. 50052. 49973. ... 50037. 50175. 49695.]
 [49243. 49867. 50322. ... 50106. 50751. 50599.]
 [49548. 49603. 50570. ... 49803. 49933. 50183.]
 ...
 [50161. 50628. 50961. ... 49622. 50732. 50074.]
 [50342. 50559. 50889. ... 49582. 50489. 50028.]
 [48973. 49888. 50068. ... 49378. 50116. 49596.]]
Соответствие результата с np.dot(): True
Время на GPU: 0.08369660377502441 секунд
Результат на CPU:
[[49437. 50052. 49973. ... 50037. 50175. 49695.]
 [49243. 49867. 50322. ... 50106. 50751. 50599.]
 [49548. 49603. 50570. ... 49803. 49933. 50183.]
 ...
 [50161. 50628. 50961. ... 49622. 50732. 50074.]
 [50342. 50559. 50889. ... 49582. 50489. 50028.]
 [48973. 49888. 50068. ... 49378. 50116. 49596.]]
Соответствие результата с np.dot(): True
Время на CPU: 5977.909497976303 секунд
